# NetCDF-to-Zarr service example usage

The scope of this notebook is to demonstrate example requests that use the NetCDF-to-Zarr Harmony backend service via `harmony-py`. Once each request has completed, the output will be accessed in-region from S3.

To run the cells that access the data in S3, this notebook will need to be run from within the AWS `us-west-2` region. One way of accomplishing this is to run the notebook within the [Openscapes 2i2c Jupyter Hub](https://openscapes.2i2c.cloud/hub).

### Authentication prerequisites:

The `harmony.Client` class will attempt to use credentials from a local `.netrc` file, located in the home directory of the filesystem where this notebook is running. This will need to contain entries for Earthdata Login (at minimum for the UAT environment):

```
machine urs.earthdata.nasa.gov
    login <prod_edl_username>
    password <prod_edl_password>

machine uat.urs.earthdata.nasa.gov
    login <uat_edl_username>
    password <uat_edl_password>
```

### Importing required packages:

The cell below imports classes and functions from various Python packages, these packages include:

* `harmony-py`: A package that allows for easy, interaction with the Harmony API that can be written directly in Python.
* `pprint`: A lightweight package that can print data structures with indentation to aid in their visual inspection.
* `s3fs`: A package that enables interactions with items stored in AWS S3.
* `matplotlib`: A package used extensively for plotting data in Python.
* `xarray`: A package that can read and write data stored in a number of formats, including NetCDF-4 and Zarr.

Further packages required by the Python environment running this notebook include:

* `netCDF4`: A package primarily used for interacting with NetCDF-4 files. This package is not explicitly imported below, but `xarray` requires this to read a Zarr store.
* `zarr`: A package that interacts with Zarr stores. Again, this package is not explicitly imported below, but `xarray` requires it to read a Zarr store.

The packages in both lists above should be installed in the environment in which this notebook is running, prior to starting the notebook. They are included by default in the Openscapes 2i2c Jupyter Hub as part of the Corn development environment.

In [ ]:
from datetime import datetime

from harmony import Client, Collection, Environment, LinkType, Request
from pprint import pprint
from s3fs import S3FileSystem
import matplotlib.pyplot as plt
import xarray as xr

### Setting up a Harmony client:

In this notebook, requests will be made against test data in the UAT environment. First an instance of the `harmony.Client` class is created, which simplifies the interactions with the Harmony API, including request submission and retrieval of results.

In [ ]:
harmony_client = Client(env=Environment.UAT)

### Setting up an S3 connection:

The `s3fs.S3FileSystem` class creates a connection to S3, such that typical filesystem commands can be used against the contents of S3 (see documentation [here](https://s3fs.readthedocs.io/en/latest/)). The same instance will be used to interact with the outputs from the requests in the notebook below. The credentials necessary to access Harmony outputs stored in AWS S3 can be generated using the `harmony.Client` class:

In [ ]:
s3_credentials = harmony_client.aws_credentials()

s3_fs = S3FileSystem(key=s3_credentials['aws_access_key_id'],
                     secret=s3_credentials['aws_secret_access_key'],
                     token=s3_credentials['aws_session_token'],
                     client_kwargs={'region_name':'us-west-2'})

## Converting a single granule:

The request below will make a request against the [Harmony Example L2 Data collection](https://cmr.uat.earthdata.nasa.gov/search/concepts/C1233860183-EEDTEST.html). Each granule is a small NetCDF-4 file, with 4 science variables and swath dimension variables.

First, a request is constructed via the `harmony.Request` class. In this request only the data collection, output format and number of granules will be specified. This request will create a Zarr store from the first granule in the collection.

The request will then be submitted to the Harmony API using the `harmony.Client` object, and URLs for the results can be retrieved once the job is completed.

In [ ]:
example_l2_collection = Collection(id='C1233860183-EEDTEST')

# Specify a request to create Zarr output for one granule in the collection:
single_granule_request = Request(collection=example_l2_collection, format='application/x-zarr',
                                 granule_id='G1233860549-EEDTEST')

# Submit the request and wait for it to complete:
single_granule_job_id = harmony_client.submit(single_granule_request)
harmony_client.wait_for_processing(single_granule_job_id, show_progress=True)

# Filter the results to only include links to resources in S3:
single_granule_result_urls = list(harmony_client.result_urls(single_granule_job_id, link_type=LinkType.s3))

### Reading calibrated data:

By default, when reading data from a Zarr store, `xarray.open_zarr` will apply the `add_offset`, `scale_factor` and `_FillValue` metadata attributes to the array values of a variable, if they are present.

In the example request, the raw data for `/blue_var` contain the following metadata attributes:

* `add_offset = 210`
* `scale_factor = 2`
* `_FillValue = 0`

In [ ]:
calibrated_dataset = xr.open_zarr(s3_fs.get_mapper(single_granule_result_urls[0]))
calibrated_dataset

### Inspecting the calibrated metadata attributes:

The dictionary below contains all the metadata attributes available for the `/blue_var` in the Zarr store when using the default options for reading the data. This variable only contains the `scale_factor`, `add_offset` and `_FillValue` metadata attributes, all of which are applied when `xarray` reads the Zarr store and automatically masks and scales the variable array values. These metadata attributes are therefore not retained after they have been applied to the data and as such the dictionary below will be empty.

In [ ]:
calibrated_dataset.blue_var.attrs

### Plotting calibrated variable values:

As noted, the `/blue_var` variable has the `scale_factor`, `add_offset` and `_FillValue` metadata attributes in the raw data. Plotting the data for this variable via `xarray` show that these values have been applied to the data by default. The data in `/blue_var` are masked, such that fill values are present at all points in the array covering land, in this case the continent of Africa. As `xarray` has already applied the stored fill value to the array, Africa is plotted in white, denoting a region where the array contains recognised fill values.

Also note the scale of the data ranging from 212 to 240. Because the raw data are integers, and `_FillValue = 0`, the lowest possible raw data value is 1, when scaled this becomes 212.

In [ ]:
plot = calibrated_dataset.blue_var.plot(x='lon', y='lat', cmap=plt.cm.coolwarm)

### Reading raw data:

The raw array values for variables can be read using `xarray.open_zarr` by specifying the `mask_and_scale` keyword argument to be `False` (it is `True` by default). This is shown below.

Note: because these attributes are not automatically applied to the values, they will be persisted as attributes in the `.attrs` dictionary of the relevant variable (e.g., `raw_dataset.blue_var.attrs`). These attributes will also now be shown in the table displayed below. Clicking on the metadata icon, 📄, for a variable will expand its entry in the table to show all available metadata attributes.

In [ ]:
raw_dataset = xr.open_zarr(s3_fs.get_mapper(single_granule_result_urls[0]), mask_and_scale=False)
raw_dataset

### Inspecting the raw metadata attributes:

The dictionary below contains the available metadata attributes for the `/blue_var` in the Zarr store, when specifying that the data should _not_ automatically apply the `scale_factor`, `add_offset` and `_FillValue` attributes. All three of these metadata attributes therefore remain available in the `attrs` dictionary below:

In [ ]:
raw_dataset.blue_var.attrs

### Plotting raw variable values:

The plot below shows the values stored by `xarray` when not automatically calibrating or masking the data using the available metadata attributes. Note the continent of Africa now has array values of 0, which corresponds to the `_FillValue` metadata attribute value, and these are not automatically recognised as fill values by `xarray`. Also note the scale of the data ranging from 0 to 15, which is the full range of the raw data values.

In [ ]:
plot = raw_dataset.blue_var.plot(x='lon', y='lat', cmap=plt.cm.coolwarm)

## Temporal Aggregation:

The NetCDF-to-Zarr service can aggregate data along a temporal dimension, such that multiple input NetCDF-4 files are combined into a single Zarr store. This is the default behaviour when requesting multiple NetCDF-4 input files within this same request. To produce an individual Zarr store for each input NetCDF-4 file instead, the request must specify `concatenate=False`. 

The temporal aggregation operation assumes that all input files are gridded (spatially and temporally), and that the spatial extents for all input NetCDF-4 files are the same. The variables being aggregated must have an associated temporal dimension that refers to the temporal grid values stored within a 1-D variable. This 1-D variable must have a `units` metadata attribute that complies with the [CF-Conventions](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.9/cf-conventions.html#time-coordinate).

The example request below will use data from the Global Precipitation Measurement (GPM) mission Integrated Multi-satellitE Retrievals for GPM (IMERG) half-hourly collection. Each granule represents a single half hour increment of time, with data stored in a three dimensional grid. The figure below shows an example GPM/IMERG granule as viewed within Panoply:

<center><img src='images/GPM_IMERG_Panoply.png'></center>

**Figure 1:** A GPM/IMERG NetCDF-4 file displayed in Panoply.

The grid dimensions for each gridded GPM/IMERG variable are $time$, $longitude$ and $latitude$, with the horizontal spatial coordinates providing whole-Earth coverage (0.1 degree resolution), while the $time$ dimension contains only a single value.

The figure below shows how the request takes 6 input NetCDF-4 files, whose grids have dimensions (1, 3600, 1800), and produces a single Zarr store with grid dimensions (6, 3600, 1800).

<center><img src='images/stacked_output.png'></center>

**Figure 2:** Left: A gridded science variable as represented in six separate NetCDF-4 input GPM/IMERG granules. These have dimensions (1, 3600, 1800). Right: The stacked variable as saved in the output Zarr store. This has dimensions (6, 3600, 1800).

The temporal aggregation will always produce a regular grid. If there are gaps in data coverage, then the temporal dimension will include a value for the missing data, but that slice will be populated with fill values.

### The temporal aggregation request:

The request below uses a temporal range to filter the GPM/IMERG collection down to 6 consecutive granules. Each of these granules corresponds to a NetCDF-4 file, each of which has a different timestamp stored in the `/Grid/time` variable. The 6 input files have consecutive time values, such that the aggregated output will form a grid with a time dimension of length 6 values, all half an hour apart.

In [ ]:
gpm_imerg = Collection(id='C1225808238-GES_DISC')

# Specify a request to create Zarr output for six granules from the GPM/IMERG collection in a specific temporal range:
temporal_aggregation_request = Request(collection=gpm_imerg,
                                       temporal={'start': datetime(2020,1, 1),
                                                 'stop': datetime(2020, 1, 1, 2, 59)},
                                       format='application/x-zarr')

# Submit the request and wait for it to complete:
temporal_aggregation_job_id = harmony_client.submit(temporal_aggregation_request)
harmony_client.wait_for_processing(temporal_aggregation_job_id, show_progress=True)

# Filter the results to only include links to resources in S3:
temporal_aggregation_result_urls = list(harmony_client.result_urls(temporal_aggregation_job_id, link_type=LinkType.s3))

### Accessing temporally aggregated results:

The following cell uses `xarray` to read the aggregated results. The variables are all contained in the `/Grid` group, so `xarray.open_zarr` specifies this group via the `group` keyword argument.

In [ ]:
aggregated_dataset = xr.open_zarr(s3_fs.get_mapper(temporal_aggregation_result_urls[0]), group='/Grid', decode_cf=True)
aggregated_dataset

### Plotting the data:

In the cell below, `xarray` plotting functionality is used to plot each of the temporal slices of data in the aggregated `/Grid/precipitationCal` variable. The output will display 6 panels in two columns. Each column corresponds to a slice in the 3-dimensional array, with the corresponding time value displayed in the title of the panel. The spatial region has been specified by `xlim` and `ylim`, zooming in on a 20° x 20° region of the southern Pacific Ocean, such that it is possible to visually compare the output at each time, showing an evolution between the array slices.

In [ ]:
aggregated_dataset.precipitationCal.plot(x='lon', y='lat', col='time', col_wrap=2, size=8, vmax=40,
                                         cmap=plt.cm.coolwarm, xlim=[-169, -149], ylim=[-47, -27])

### Should I temporally aggregate my collection?

Concatenation into a single Zarr store is now the default behaviour. To retrieve a single Zarr store per input NetCDF-4 granule, the `concatenate=false` query parameter is required.

When to _not_ concatenate:

* Different input NetCDF-4 files cover different spatial extents (a future extension).
* The input NetCDF-4 files do not include time as a grid dimension, e.g.: $(longitude, latitude)$ rather than $(time, longitude, latitude)$.
* Large input data volume (either many or very large input NetCDF-4 files).